In [73]:
def get_stock_price(company):
    """
    This function takes a company name as input and returns the current stock price.
    Note: This is a placeholder function. In a real-world scenario, you would use an API to get the stock price.
    """
    # Placeholder for actual stock price retrieval logic
    stock_prices ={
        'symbol': 'GM', 
        'price': 43.09
    }
    
    return stock_prices

def div_by_price(price, symbol):
    """
    This function takes a price and a stock symbol as input and returns the division of the price by the stock price.
    """
    stock_info = get_stock_price(symbol)
    stock_price = float(stock_info['price'])
    
    if stock_price == 0:
        raise ValueError("Stock price cannot be zero.")
    
    return price / stock_price

In [91]:
tool_definition = {
    "name": "get_stock_price",
    "description": "Retrieves the current stock price for a given company",
    "input_schema": {
        "type": "object",
        "properties": {
            "company": {
                "type": "string",
                "description": "The company name to fetch stock data for"
            }
        },
        "required": ["company"]
    }
}

tool_definition_div_by_price = {
    "name": "divide_numbers",
    "description": "Divides two numbers and returns the quotient.",
    "input_schema": {
        "type": "object",
        "properties": {
            "number1": {
                "type": "number",
                "description": "The first number (dividend)."
            },
            "number2": {
                "type": "number",
                "description": "The second number (divisor)."
            }
        },
        "required": ["number1", "number2"]
    }
}

In [30]:
import anthropic
import pandas as pd

client = anthropic.Anthropic()
history = []

MODEL_NAME = "claude-3-7-sonnet-20250219"

# 1. Hàm đọc CSV và tạo DataFrame
def get_dataframe_revit(file_path):
    try:
        df = pd.read_csv(file_path, encoding='utf-8')
        return df
    except Exception as e:
        print(f"Error reading CSV file: {e}")
        return None

# 2. Hàm lấy danh sách các cột trong DataFrame
def get_all_columns(df: pd.DataFrame) -> list:
    """
    Returns all columns in the DataFrame.
    
    Args:
        df (pd.DataFrame): Input DataFrame.
    Returns:
        list: List of all column names.
    """
    return df.columns.tolist()

# 3. Hàm lấy các giá trị duy nhất trong một cột
def get_values_unique_in_column(df: pd.DataFrame, col: str) -> list:
    """
    Get unique values in a column of a DataFrame.
    
    Args:
        df (pd.DataFrame): The DataFrame to query.
        col (str): The column name to get unique values from.
    
    Returns:
        list: A list of unique values in the specified column.
    """
    return df[col].unique().tolist()

# 
def get_len_values_unique_in_column(df: pd.DataFrame, col: str) -> str:
    return str(len(df[col].unique().tolist()))

# 4. Hàm tìm kiếm giá trị trong cột
def search_value_in_column(df: pd.DataFrame, col_name: str, value_search: str):
    """
    Args:
        df (pd.DataFrame): dataset revit API.
        col_name (str): name column in DataFrame.
        value_search (str): value to search.
    
    Returns:
        list: A list of elements that contain the value being searched.
    """
    category_column = [col for col in df.columns if col_name.lower() in col.lower()]
    if not category_column:
        return []
    values = [v for v in df[category_column[0]] if value_search.lower() in v.lower()]
    return values


# 5. Định nghĩa các tools cho Model.
tools = [
    {
        "name": "get_all_columns",
        "description": "Returns all column names and the quantity of columns in the data, The column name may be external_id, ElementId, Name, Category, CategoryId, Workset, Edited by ...",
        "input_schema": {
            "type": "object",
            "properties": {
                "column_names": {
                    "type": "string",
                    "description": "Columns name may be: dbId, external_id, ElementId, Name, Category, CategoryId, Workset, Edited by..."
                }
            },
            "required": ["column_names"]
        },
    },
    {
        "name": "get_values_unique_in_column",
        "description": "The column name may be external_id, ElementId, Name, Category, CategoryId, Workset, Edited by ...",
        "input_schema": {
            "type": "object",
            "properties": {
                "column_name": {
                    "type": "string",
                    "description": "The column name may be external_id, ElementId, Name, Category, CategoryId, Workset, Edited by ..."
                },
            },
            "required": ["column_name"]
        },
    },
    {
        "name": "get_len_values_unique_in_column",
        "description": "The column name may be external_id, ElementId, Name, Category, CategoryId, Workset, Edited by ...",
        "input_schema": {
            "type": "object",
            "properties": {
                "len_values": {
                    "type": "string",
                    "description": "Quantity of the only values ​​in the column"
                },
            },
            "required": ["len_values"]
        },
    }

]

# 6. Hàm gửi câu hỏi tới Model
def get_response(question: str, df: pd.DataFrame):
    # Nếu lịch sử rỗng, thêm context của DataFrame.
    if len(history) == 0:
        col_names = ", ".join(df.columns.to_list())
        history.append({
            "role": "user",
            "content":
                f"""I have a DataFrame from the path data.csv with columns names: {col_names}.\n\n{question}\n"""
        })
    else:
        if history[-1]['content'] != question:
            history.append({
                "role": "user", 
                "content": question
            })
    
    return client.messages.create(
        model=MODEL_NAME,
        max_tokens=100,
        tools=tools,
        messages=history,    
    )

# 7. Hàm thực hiện công cụ dựa trên tên tool và đầu vào
def execute_tool(tool_name, tool_input):
    path = 'data.csv'
    df = get_dataframe_revit(path)
    print(f"execute_tool: DataFrame loaded from path: {path}")
    
    if tool_name == "get_all_columns":
        return get_all_columns(df)
    elif tool_name == "get_values_unique_in_column":
        print("Tool input: ", tool_input)
        return get_values_unique_in_column(df, tool_input['column_name'])
    elif tool_name == "get_len_values_unique_in_column":
        print("Tool input: ", tool_input)
        return get_len_values_unique_in_column(df, tool_input['column_name'])
    else:
        raise ValueError(f"Unknown tool: {tool_name}")

# 8. Hàm xử lý và in phản hồi từ model
def execute_response(response: object):
    print("*" * 100)
    print(response)
    print("*" * 100)
    for tool in response.content:
        if tool.type == "text":
            print("Execute_response:", tool.text)
        elif tool.type == "tool_use":
            tool_use = tool.name
            print(f"Tool used: {tool_use}")
            # Thực thi tool dựa trên tên và input đi kèm
            res = execute_tool(tool_use, tool.input)
            if isinstance(res, list):
                # Nếu kết quả là list, chuyển thành chuỗi hiển thị
                result_str = ", ".join(str(item) for item in res)
                print(f"\nTool output: {result_str}")
                history.append({'role': "assistant", 'content': f"Result tool {tool_use}: {result_str}"})
            else:
                history.append({'role': "assistant", 'content': f"Result tool {tool_use}: {str(res)}"})
        else:
            raise ValueError(f"Lỗi xác định tool trong phản hồi: {response}")
        print()

# 9. Sử dụng: Đọc dữ liệu và gửi câu hỏi
df = pd.read_csv('data.csv')
question = "Get all column name in data?"
initial_response = get_response(question, df)
print(f"Question: {question}")
execute_response(initial_response)

Question: Get all column name in data?
****************************************************************************************************
Message(id='msg_01QTEwRUNX95Lb1bHHiFkitQ', content=[TextBlock(citations=None, text="I'll help you get all the column names from your DataFrame. Let me use the appropriate tool to retrieve this information.", type='text'), ToolUseBlock(id='toolu_01MrPQGKwSEfG8Dnw8VqCoH6', input={'column_names': 'all'}, name='get_all_columns', type='tool_use')], model='claude-3-7-sonnet-20250219', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=Usage(cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=3981, output_tokens=82))
****************************************************************************************************
Execute_response: I'll help you get all the column names from your DataFrame. Let me use the appropriate tool to retrieve this information.

Tool used: get_all_columns
execute_tool: DataFrame

In [4]:
history

[{'role': 'user',
  'content': "I have a DataFrame from the path data.csv with columns names: Unnamed: 0, dbId, external_id, ElementId, Name, Category, CategoryId, Workset, Edited by, Image, Keynote, Export to IFC, Export to IFC As, IFC Predefined Type, IfcGUID, Model, Manufacturer, Comments, URL, Description, Cost, Mark, Color, Transparency, Glow, Smoothness, Shininess, Material Type, Class, Subclass, Behavior, Young modulus X, Young modulus Y, Young modulus Z, Poisson ratio X, Poisson ratio Y, Poisson ratio Z, Shear modulus X, Shear modulus Y, Shear modulus Z, Thermal expansion coefficient X, Thermal expansion coefficient Y, Thermal expansion coefficient Z, Unit weight, Minimum yield stress, Minimum tensile strength, Thermally Treated, Reduction factor for shear, Concrete compression, Lightweight, Shear strength modification, Young modulus, Poisson ratio, Shear modulus, Thermal expansion coefficient, Species, Grade, Bending, Compression parallel to grain, Compression perpendicular to

In [33]:
df = pd.read_csv('data.csv')
question = "How many Family Name in data"
initial_response = get_response(question, df)
print(f"Question: {question}")
execute_response(initial_response)

Question: How many Family Name in data
****************************************************************************************************
Message(id='msg_01XqskxAy2fqkXbaYbyXJyGd', content=[TextBlock(citations=None, text='Result tool get_len_values_unique_in_column: 599', type='text')], model='claude-3-7-sonnet-20250219', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=Usage(cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=15761, output_tokens=19))
****************************************************************************************************
Execute_response: Result tool get_len_values_unique_in_column: 599



history

In [26]:
initial_response

Message(id='msg_0121vdQ6qNJrqSQSSDAkxhHC', content=[TextBlock(citations=None, text='To answer your question about how many different Family Name types exist in the data, I need to count the unique values in the "Family Name" column.', type='text'), ToolUseBlock(id='toolu_01BSYhfU7PC9Q4SPVbREkj3C', input={'column_name': 'Family Name'}, name='get_values_unique_in_column', type='tool_use')], model='claude-3-7-sonnet-20250219', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=Usage(cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=11416, output_tokens=94))

In [ ]:
initial_response

In [25]:
len(history), history

(6,
 [{'role': 'user',
   'content': "I have a DataFrame from the path data.csv with columns names: Unnamed: 0, dbId, external_id, ElementId, Name, Category, CategoryId, Workset, Edited by, Image, Keynote, Export to IFC, Export to IFC As, IFC Predefined Type, IfcGUID, Model, Manufacturer, Comments, URL, Description, Cost, Mark, Color, Transparency, Glow, Smoothness, Shininess, Material Type, Class, Subclass, Behavior, Young modulus X, Young modulus Y, Young modulus Z, Poisson ratio X, Poisson ratio Y, Poisson ratio Z, Shear modulus X, Shear modulus Y, Shear modulus Z, Thermal expansion coefficient X, Thermal expansion coefficient Y, Thermal expansion coefficient Z, Unit weight, Minimum yield stress, Minimum tensile strength, Thermally Treated, Reduction factor for shear, Concrete compression, Lightweight, Shear strength modification, Young modulus, Poisson ratio, Shear modulus, Thermal expansion coefficient, Species, Grade, Bending, Compression parallel to grain, Compression perpendicu

In [56]:
df = pd.read_csv('data.csv')
question = "What is my previous question?"
initial_response = get_response(question, df)
print(f"Question: {question}")
print(f"====================================================================\n")
execute_response(initial_response)

Question: What is my previous question?

Excute_response:  Your previous question was "how many types 'Family Name' in data"



In [61]:
print(len(history))

df = pd.read_csv('data.csv')
question = "How many walls in data"
initial_response = get_response(question, df)
print(f"Question: {question}")
print(f"====================================================================\n")
execute_response(initial_response)

12
Question: How many walls in data

Excute_response:  I need to check the data to answer your question about how many walls are in the data. Let me retrieve that information for you.

Tool used: get_values_unique_in_column
excute_tool: DataFrame loaded from path: data.csv
Tool input:  {'input_name': 'Category'}



In [62]:
history

[{'role': 'user',
  'content': 'I have a DataFrame from the path of data.csv\nwith columns names: [\'Unnamed: 0\', \'dbId\', \'external_id\', \'ElementId\', \'Name\', \'Category\', \'CategoryId\', \'Workset\', \'Edited by\', \'Image\', \'Keynote\', \'Export to IFC\', \'Export to IFC As\', \'IFC Predefined Type\', \'IfcGUID\', \'Model\', \'Manufacturer\', \'Comments\', \'URL\', \'Description\', \'Cost\', \'Mark\', \'Color\', \'Transparency\', \'Glow\', \'Smoothness\', \'Shininess\', \'Material Type\', \'Class\', \'Subclass\', \'Behavior\', \'Young modulus X\', \'Young modulus Y\', \'Young modulus Z\', \'Poisson ratio X\', \'Poisson ratio Y\', \'Poisson ratio Z\', \'Shear modulus X\', \'Shear modulus Y\', \'Shear modulus Z\', \'Thermal expansion coefficient X\', \'Thermal expansion coefficient Y\', \'Thermal expansion coefficient Z\', \'Unit weight\', \'Minimum yield stress\', \'Minimum tensile strength\', \'Thermally Treated\', \'Reduction factor for shear\', \'Concrete compression\', \

In [ ]:
import anthropic
import yfinance as yf
import pandas as pd

client = anthropic.Anthropic()
MODEL_NAME = "claude-3-7-sonnet-20250219"

# 1. get stock price function / Tool
def get_data_csv(path):
    """
    This function takes a CSV file path as input and returns the data as a pandas DataFrame.
    """
    try:
        data = pd.read_csv(path, encoding='utf-8')
        return data
    except Exception as e:
        print(f"Error reading CSV file: {e}")
        return None

# 2. Tool definition
tools = [
    {
        "name": "get_data_revit",
        "description": "Get all coponents from Revit data",
        "input_schema": {
            "type": "object",
            "properties": {
                "ticker_symbol": {
                    "type": "string",
                    "description": "The stock ticker symbol, e.g., 'AAPL' for Apple Inc."
                }
            },
            "required": ["ticker_symbol"],
        }
    }
]

# 3. Ask the model to use the tool
initial_response = client.messages.create(
    model="claude-3-7-sonnet-20250219",
    max_tokens=1024,
    tools=tools,
    messages=[
        {"role": "user", "content": "What is the current stock price of AAPL?"},
    ],
)
print(f"Stop Reason: {initial_response.stop_reason}")
print(f"Initial Response: {initial_response.content}")
print(initial_response)

# 4. Parse the tool name and run the stock price tool
def process_tool_call(tool_name, tool_input):
    if tool_name == "get_stock_price":
        return get_stock_price(tool_input["ticker_symbol"])
    
if initial_response.stop_reason == "tool_use":
    tool_use = next(block for block in initial_response.content if block.type == "tool_use")
    tool_name = tool_use.name
    tool_input = tool_use.input
    
    print(f"Tool Name: {tool_name}") # get_stock_price
    print(f"Tool Input: {tool_input}") # {'ticker_symbol': 'AAPL'}
    
    tool_result = process_tool_call(tool_name, tool_input)
    
    print(f"Tool Result: {tool_result}")
    # # 5. Send the result back to the assistant
    # print(initial_response.content)
    # response = client.messages.create(
    #     model="claude-3-7-sonnet-20250219",
    #     max_tokens=1024,
    #     tools=tools,
    #     tool_choice={"type": "auto"},
    #     messages=[
    #         {"role": "assistant", "content": tool_result},
    #     ],
    # )
else:
    response = initial_response
    
final_response = next(
    (block.text for block in response.content if block.type == "text"), None
)
print(f"Final Response: {final_response}")

Stop Reason: tool_use
Initial Response: [TextBlock(citations=None, text="I'll help you find the current stock price of AAPL (Apple Inc.). Let me retrieve that information for you.", type='text'), ToolUseBlock(id='toolu_0144ZpK2qZrQb3iKdotuSoAD', input={'ticker_symbol': 'AAPL'}, name='get_data_revit', type='tool_use')]
Message(id='msg_01Gd7Yuxvx8hwFzp7Vf2fcsf', content=[TextBlock(citations=None, text="I'll help you find the current stock price of AAPL (Apple Inc.). Let me retrieve that information for you.", type='text'), ToolUseBlock(id='toolu_0144ZpK2qZrQb3iKdotuSoAD', input={'ticker_symbol': 'AAPL'}, name='get_data_revit', type='tool_use')], model='claude-3-7-sonnet-20250219', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=Usage(cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=420, output_tokens=88))
Tool Name: get_data_revit
Tool Input: {'ticker_symbol': 'AAPL'}
Tool Result: None


AttributeError: 'Completion' object has no attribute 'content'